In [9]:
a = ['1','2','3','1']

In [10]:
from collections import Counter
c = Counter(a)
c

Counter({'1': 2, '2': 1, '3': 1})

In [11]:
for a,b in c.items():
    print (a,b)

1 2
2 1
3 1


In [12]:
from utils import *
from collections import Counter


row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
# TODO: Update the unit list to add the new diagonal units
diagonal_units =  [[rows[int(c)-1]+c for c in cols], [rows[9-int(c)]+c for c in cols[::-1]]]
unitlist = unitlist+diagonal_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


def naked_twins(values):
    """Eliminate values using the naked twins strategy.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked twins eliminated from peers

    Notes
    -----
    Your solution can either process all pairs of naked twins from the input once,
    or it can continue processing pairs of naked twins until there are no such
    pairs remaining -- the project assistant test suite will accept either
    convention. However, it will not accept code that does not process all pairs
    of naked twins from the original input. (For example, if you start processing
    pairs of twins and eliminate another pair of twins before the second pair
    is processed then your code will fail the PA test suite.)

    The first convention is preferred for consistency with the other strategies,
    and because it is simpler (since the reduce_puzzle function already calls this
    strategy repeatedly).
    """
    # TODO: Implement this function!

    # Find all instances of naked twins
    twins_list = []
    for unit in unitlist:
        vals = [ values[box] for box in unit if len(values[box])==2]
        counter = Counter(vals)
        for val, count in counter.items():
            if count == 2: 
                temp = [box for box in unit if values[box] == val]
                twins_list.append(set(temp))

    new_vals = values.copy()
    # Eliminate the naked twins as possibilities for their peers
    for twins in twins_list:
        box1, box2 = twins
        common_peer = peers[box1] & peers[box2]
        for peer in common_peer:
            temp_val = values[peer].replace(values[box1][0],'').replace(values[box1][1],'')
            new_vals = assign_value(new_vals, peer, temp_val)
    return new_vals


def eliminate(values):
    """Apply the eliminate strategy to a Sudoku puzzle

    The eliminate strategy says that if a box has a value assigned, then none
    of the peers of that box can have the same value.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the assigned values eliminated from peers
    """
    # TODO: Copy your code from the classroom to complete this function
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values


def only_choice(values):
    """Apply the only choice strategy to a Sudoku puzzle

    The only choice strategy says that if only one box in a unit allows a certain
    digit, then that box must be assigned that digit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with all single-valued boxes assigned

    Notes
    -----
    You should be able to complete this function by copying your code from the classroom
    """
    # TODO: Copy your code from the classroom to complete this function
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    """Reduce a Sudoku puzzle by repeatedly applying all constraint strategies

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary after continued application of the constraint strategies
        no longer produces any changes, or False if the puzzle is unsolvable 
    """
    # TODO: Copy your code from the classroom and modify it to complete this function
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


def search(values):
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles
    that cannot be solved by repeated reduction alone.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary with all boxes assigned or False

    Notes
    -----
    You should be able to complete this function by copying your code from the classroom
    and extending it to call the naked twins strategy.
    """
    # TODO: Copy your code from the classroom to complete this function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt


def solve(grid):
    """Find the solution to a Sudoku puzzle using search and constraint propagation

    Parameters
    ----------
    grid(string)
        a string representing a sudoku grid.
        
        Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

    Returns
    -------
    dict or False
        The dictionary representation of the final sudoku grid or False if no solution exists.
    """
    values = grid2values(grid)
    values = search(values)
    return values


if __name__ == "__main__":
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid2values(diag_sudoku_grid))
    result = solve(diag_sudoku_grid)
    display(result)

    try:
        import PySudoku
        PySudoku.play(grid2values(diag_sudoku_grid), result, history)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [4]:
from utils import *
from collections import Counter


row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units

# TODO: Update the unit list to add the new diagonal units
diagonal_units =  [[rows[int(c)-1]+c for c in cols], [rows[9-int(c)]+c for c in cols[::-1]]]
unitlist = unitlist+diagonal_units

units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)


def find_twins(values, unit):
    """
    Find naked twins which have exactly two possible values left.
    Args:
        unit_values(array): a list of box units {square: string}
    Returns:
        naked__values(array): a list of naked_twins
    """
    unit_values = [values[box] for box in unit]
    # Find naked twin values from the current unit
    naked_values = [n for n in unit_values if unit_values.count(n) == 2 and len(n) == 2]
    return naked_values

def elminate_twins(values, naked_values, unit):
    """
    Eliminate naked twins from peers.
    """
    for val in naked_values:
        for ch in val:
            for box in unit:
                if values[box] != val:
                    values = assign_value(values, box, values[box].replace(ch, ''))
    return values


def naked_twins(values):
    """Eliminate values using the naked twins strategy.
    Args:
        values(dict): a dictionary of the form {'box_name': '123456789', ...}
    Returns:
        the values dictionary with the naked twins eliminated from peers.
    """
    for unit in unitlist:
        # Find possible naked twins
        naked_values = find_twins(values, unit)
        # Elminate naked twin values from peers
        values = elminate_twins(values, naked_values, unit)
    return values


def eliminate(values):
    """Apply the eliminate strategy to a Sudoku puzzle

    The eliminate strategy says that if a box has a value assigned, then none
    of the peers of that box can have the same value.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the assigned values eliminated from peers
    """
    # TODO: Copy your code from the classroom to complete this function
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values


def only_choice(values):
    """Apply the only choice strategy to a Sudoku puzzle

    The only choice strategy says that if only one box in a unit allows a certain
    digit, then that box must be assigned that digit.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with all single-valued boxes assigned

    Notes
    -----
    You should be able to complete this function by copying your code from the classroom
    """
    # TODO: Copy your code from the classroom to complete this function
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    """Reduce a Sudoku puzzle by repeatedly applying all constraint strategies

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary after continued application of the constraint strategies
        no longer produces any changes, or False if the puzzle is unsolvable 
    """
    # TODO: Copy your code from the classroom and modify it to complete this function
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values


def search(values):
    """Apply depth first search to solve Sudoku puzzles in order to solve puzzles
    that cannot be solved by repeated reduction alone.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict or False
        The values dictionary with all boxes assigned or False

    Notes
    -----
    You should be able to complete this function by copying your code from the classroom
    and extending it to call the naked twins strategy.
    """
    # TODO: Copy your code from the classroom to complete this function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt


def solve(grid):
    """Find the solution to a Sudoku puzzle using search and constraint propagation

    Parameters
    ----------
    grid(string)
        a string representing a sudoku grid.
        
        Ex. '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'

    Returns
    -------
    dict or False
        The dictionary representation of the final sudoku grid or False if no solution exists.
    """
    values = grid2values(grid)
    values = search(values)
    return values


if __name__ == "__main__":
    diag_sudoku_grid = '2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3'
    display(grid2values(diag_sudoku_grid))
    result = solve(diag_sudoku_grid)
    display(result)

    try:
        import PySudoku
        PySudoku.play(grid2values(diag_sudoku_grid), result, history)

    except SystemExit:
        pass
    except:
        print('We could not visualize your board due to a pygame issue. Not a problem! It is not a requirement.')


    2     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789     6     |    2     123456789 123456789 
123456789 123456789     1     |123456789 123456789 123456789 |123456789     7     123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     6     |123456789 123456789     8     |123456789 123456789 123456789 
    3     123456789 123456789 |123456789     9     123456789 |123456789 123456789     7     
123456789 123456789 123456789 |    6     123456789 123456789 |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789     4     123456789 |123456789 123456789 123456789 |    8     123456789 123456789 
123456789 123456789     5     |    2     123456789 123456789 |123456789 123456789 123456789 
123456789 123456789 123456789 |123456789 123456789 123456789 |12345678

In [29]:
def find_twins(values, unit):
    unit_values = [values[box] for box in unit]
    # Find naked twin values from the current unit
    naked_values = [n for n in unit_values if unit_values.count(n) == 2 and len(n) == 2]
    #if naked_values:
    #    return [naked_values[0]]

    return naked_values

def elminate_twins(values, naked_values, unit):
    """
    Eliminate naked twins from peers.
    """
    for val in naked_values:
        for ch in val:
            for box in unit:
                if values[box] != val:
                    values = assign_value(values, box, values[box].replace(ch, ''))
    return values

twin_list = []
def naked_twins(values):
    for unit in unitlist:
        # Find possible naked twins
        naked_values = find_twins(values, unit)
        twin_list.append(naked_values)
        # Elminate naked twin values from peers
        values = elminate_twins(values, naked_values, unit)
    return values, twin_list

In [33]:
def find_twins(values, unit):
    unit_values = [values[box] for box in unit]
    # Find naked twin values from the current unit
    naked_values = [n for n in unit_values if unit_values.count(n) == 2 and len(n) == 2]
    #if naked_values:
    #    return [naked_values[0]]

    return naked_values

def elminate_twins(values, naked_values, unit):

    for val in naked_values:
        for box in unit:
            if values[box] != val :
                values = assign_value(values, box, values[box].replace(val[0], '').replace(val[1],''))
    return values

twin_list = []
def naked_twins(values):
    for unit in unitlist:
        # Find possible naked twins
        naked_values = find_twins(values, unit)
        twin_list.append(naked_values)
        # Elminate naked twin values from peers
        values = elminate_twins(values, naked_values, unit)
    return values, twin_list

In [56]:
def naked_twins1(values):

    twins_list = []
    for unit in unitlist:
        vals = [ values[box] for box in unit if len(values[box])==2]
        counter = Counter(vals)
        for val, count in counter.items():
            if count == 2: 
                temp = [box for box in unit if values[box] == val]
                twins_list.append(set(temp))

    new_vals = values.copy()
    # Eliminate the naked twins as possibilities for their peers
    twins_list = twins_list
    for twins in twins_list:
        box1, box2 = twins
        common_peer = peers[box1] & peers[box2]
        for peer in common_peer:
            if peer != box1 and peer != box2 :
                temp_val = values[peer].replace(values[box1][0],'').replace(values[box1][1],'')
                values = assign_value(new_vals, peer, temp_val)
    return new_vals, twins_list

In [57]:
a = {"H7": "2345678", "H3": "1234569", "I4": "345678", "D7": "47", "D2":
"9", "C9": "124578", "I6": "23678", "F5": "37", "I5": "2345678", "I3":
"23456", "E1": "567", "H8": "345678", "D9": "47", "G6": "1236789",
"C3": "1234569", "A3": "7", "F4": "9", "F2": "24", "I8": "345678",
"E4": "2", "F9": "6", "C1": "2345689", "F8": "58", "D4": "1", "B2":
"1234568", "B8": "3456789", "C2": "1234568", "H1": "23456789", "H4":
"345678", "A1": "2345689", "I9": "9", "E6": "4", "G3": "1234569",
"G7": "2345678", "E3": "56", "G8": "345678", "H2": "12345678", "C7":
"2345678", "F7": "58", "I2": "2345678", "A9": "2458", "A2": "234568",
"D6": "5", "G5": "23456789", "B9": "124578", "E8": "1", "F1": "1",
"A8": "345689", "F6": "37", "G2": "12345678", "C6": "236789", "B5":
"23456789", "F3": "24", "D1": "36", "B7": "2345678", "I1": "2345678",
"E2": "567", "E7": "9", "A5": "1", "D8": "2", "D3": "8", "B3":
"1234569", "H5": "23456789", "A7": "234568", "G1": "23456789", "D5":
"36", "B1": "2345689", "C4": "345678", "C5": "23456789", "E5": "678",
"A6": "23689", "C8": "3456789", "G4": "345678", "E9": "3", "A4":
"34568", "H9": "24578", "B4": "345678", "I7": "1", "B6": "236789",
"G9": "24578", "H6": "1236789"}

In [65]:
c, d = naked_twins1(a)

In [66]:
a['D1'], a['D5'], a['D7'], a['D9'], a['F2'], a['F3'], a['F5'], a['F6']

('36', '6', '47', '47', '24', '24', '37', '37')

In [70]:
peers['D1'] & peers['D5']

{'D2', 'D3', 'D4', 'D6', 'D7', 'D8', 'D9'}

In [54]:
d

[{'D1', 'D5'},
 {'D7', 'D9'},
 {'F2', 'F3'},
 {'F5', 'F6'},
 {'F7', 'F8'},
 {'F2', 'F3'},
 {'F5', 'F6'},
 {'D7', 'D9'},
 {'F7', 'F8'}]

In [63]:
d

[[],
 [],
 [],
 ['36', '36', '47', '47'],
 [],
 ['24', '24', '37', '37', '58', '58'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['24', '24'],
 ['37', '37'],
 ['47', '47', '58', '58'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['36', '36', '47', '47'],
 [],
 ['24', '24', '37', '37', '58', '58'],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['24', '24'],
 ['37', '37'],
 ['47', '47', '58', '58'],
 [],
 [],
 [],
 [],
 []]

In [102]:
naked_twins(a)

{'A1': '2345689',
 'A2': '234568',
 'A3': '7',
 'A4': '34568',
 'A5': '1',
 'A6': '23689',
 'A7': '234568',
 'A8': '345689',
 'A9': '2458',
 'B1': '2345689',
 'B2': '1234568',
 'B3': '1234569',
 'B4': '345678',
 'B5': '23456789',
 'B6': '236789',
 'B7': '2345678',
 'B8': '3456789',
 'B9': '124578',
 'C1': '2345689',
 'C2': '1234568',
 'C3': '1234569',
 'C4': '345678',
 'C5': '23456789',
 'C6': '236789',
 'C7': '2345678',
 'C8': '3456789',
 'C9': '124578',
 'D1': '36',
 'D2': '9',
 'D3': '8',
 'D4': '1',
 'D5': '6',
 'D6': '5',
 'D7': '47',
 'D8': '2',
 'D9': '47',
 'E1': '567',
 'E2': '567',
 'E3': '56',
 'E4': '2',
 'E5': '68',
 'E6': '4',
 'E7': '9',
 'E8': '1',
 'E9': '3',
 'F1': '1',
 'F2': '24',
 'F3': '24',
 'F4': '9',
 'F5': '37',
 'F6': '37',
 'F7': '58',
 'F8': '58',
 'F9': '6',
 'G1': '23456789',
 'G2': '12345678',
 'G3': '1234569',
 'G4': '345678',
 'G5': '23456789',
 'G6': '1236789',
 'G7': '2345678',
 'G8': '345678',
 'G9': '24578',
 'H1': '23456789',
 'H2': '12345678',
 '

In [104]:
def naked_twins(values):
    """Eliminate values using the naked twins strategy.

    Parameters
    ----------
    values(dict)
        a dictionary of the form {'box_name': '123456789', ...}

    Returns
    -------
    dict
        The values dictionary with the naked twins eliminated from peers

    Notes
    -----
    Your solution can either process all pairs of naked twins from the input once,
    or it can continue processing pairs of naked twins until there are no such
    pairs remaining -- the project assistant test suite will accept either
    convention. However, it will not accept code that does not process all pairs
    of naked twins from the original input. (For example, if you start processing
    pairs of twins and eliminate another pair of twins before the second pair
    is processed then your code will fail the PA test suite.)

    The first convention is preferred for consistency with the other strategies,
    and because it is simpler (since the reduce_puzzle function already calls this
    strategy repeatedly).
    """
    # TODO: Implement this function!

    # Find all instances of naked twins
    twins_list = []
    for unit in unitlist:
        vals = [ values[box] for box in unit if len(values[box])==2]
        counter = Counter(vals)
        for val, count in counter.items():
            if count == 2: 
                temp = [box for box in unit if values[box] == val]
                twins_list.append(set(temp))

    new_vals = values.copy()
    # Eliminate the naked twins as possibilities for their peers
    for twins in twins_list:
        box1, box2 = twins
        common_peer = peers[box1] & peers[box2]
        for peer in common_peer:
            temp_val = values[peer].replace(values[box1][0],'').replace(values[box1][1],'')
            new_vals = assign_value(new_vals, peer, temp_val)
    return new_vals

In [105]:
b = {"F6": "37", "G9": "24578", "I6": "23678", "A4": "34568", "E4": "2",
"I7": "1", "C4": "345678", "B7": "2345678", "C7": "2345678", "D5":
"36", "G2": "12345678", "F9": "6", "A9": "2458", "F7": "58", "B9":
"124578", "I3": "23456", "A8": "345689", "G3": "1234569", "A5": "1",
"D2": "9", "F2": "24", "H6": "1236789", "G1": "23456789", "I9": "9",
"F3": "24", "I1": "2345678", "H3": "1234569", "B1": "2345689", "I2":
"2345678", "G5": "23456789", "C1": "2345689", "H8": "345678", "E9":
"3", "H9": "24578", "C9": "124578", "D9": "47", "G6": "1236789", "D3":
"8", "F8": "58", "F1": "1", "C6": "236789", "C2": "1234568", "D7":
"47", "B3": "1234569", "C5": "23456789", "F4": "9", "H5": "23456789",
"E1": "567", "B8": "3456789", "C3": "1234569", "D6": "5", "E5": "678",
"A7": "234568", "A6": "23689", "E8": "1", "E2": "567", "H7":
"2345678", "A1": "2345689", "H2": "12345678", "B4": "345678", "I5":
"2345678", "B2": "1234568", "E7": "9", "I4": "345678", "D8": "2",
"D4": "1", "F5": "37", "A3": "7", "G7": "2345678", "H4": "345678",
"E3": "56", "H1": "23456789", "I8": "345678", "B5": "23456789", "E6":
"4", "C8": "3456789", "D1": "36", "G8": "345678", "A2": "234568",
"B6": "236789", "G4": "345678"}

naked_twins(b)

{'A1': '2345689',
 'A2': '234568',
 'A3': '7',
 'A4': '34568',
 'A5': '1',
 'A6': '23689',
 'A7': '234568',
 'A8': '345689',
 'A9': '2458',
 'B1': '2345689',
 'B2': '1234568',
 'B3': '1234569',
 'B4': '345678',
 'B5': '23456789',
 'B6': '236789',
 'B7': '2345678',
 'B8': '3456789',
 'B9': '124578',
 'C1': '2345689',
 'C2': '1234568',
 'C3': '1234569',
 'C4': '345678',
 'C5': '23456789',
 'C6': '236789',
 'C7': '2345678',
 'C8': '3456789',
 'C9': '124578',
 'D1': '36',
 'D2': '9',
 'D3': '8',
 'D4': '1',
 'D5': '36',
 'D6': '5',
 'D7': '47',
 'D8': '2',
 'D9': '47',
 'E1': '567',
 'E2': '567',
 'E3': '56',
 'E4': '2',
 'E5': '68',
 'E6': '4',
 'E7': '9',
 'E8': '1',
 'E9': '3',
 'F1': '1',
 'F2': '24',
 'F3': '24',
 'F4': '9',
 'F5': '37',
 'F6': '37',
 'F7': '58',
 'F8': '58',
 'F9': '6',
 'G1': '23456789',
 'G2': '12345678',
 'G3': '1234569',
 'G4': '345678',
 'G5': '23456789',
 'G6': '1236789',
 'G7': '2345678',
 'G8': '345678',
 'G9': '24578',
 'H1': '23456789',
 'H2': '12345678',
 